# MRE: Federal CTC Expansion Reduces MA State Tax

This notebook demonstrates how federal CTC expansion can reduce Massachusetts state tax by triggering itemization when federal tax is the same either way.

In [1]:
from policyengine_us import Simulation
from policyengine_us.model_api import *

## Define a tax unit where itemization becomes neutral with CTC

In [2]:
# Create a family in MA with specific characteristics
# Key: income and deductions calibrated so CTC makes itemization neutral
situation = {
    "people": {
        "parent1": {
            "age": {2026: 45},
            "employment_income": {2026: 22_500},
            "medical_out_of_pocket_expenses": {2026: 3_900},
        },
        "parent2": {
            "age": {2026: 43},
        },
        "child1": {"age": {2026: 13}},
        "child2": {"age": {2026: 12}},
        "child3": {"age": {2026: 6}},
        "child4": {"age": {2026: 4}},
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
            "state_income_tax": {2026: 900},
        }
    },
    "households": {
        "household": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
            "state_code": {2026: "MA"},
        }
    },
}

## Scenario 1: Without CTC Expansion

In [3]:
# Reform stack WITHOUT CTC (simulating TCJA extension baseline)
reform_no_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48_900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
}, country_id="us")

sim_no_ctc = Simulation(situation=situation, reform=reform_no_ctc)

print("WITHOUT CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_no_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_no_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_no_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_no_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

WITHOUT CTC EXPANSION:


  Federal itemizes: False
  Federal tax (itemizing): $-10,273.75
  Federal tax (standard): $-11,137.00
  MA state tax: $-4,394.80
  MA Part B exemption: $12,800.00


## Scenario 2: With CTC Expansion

In [4]:
# Add CTC expansion
reform_with_ctc = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48_900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    # CTC expansion parameters
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2_200},
    "gov.irs.credits.ctc.refundable.individual_max": {"2026-01-01": 1_700},
}, country_id="us")

sim_with_ctc = Simulation(situation=situation, reform=reform_with_ctc)

print("WITH CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_with_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_with_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_with_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_with_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

WITH CTC EXPANSION:


  Federal itemizes: False
  Federal tax (itemizing): $-11,137.00
  Federal tax (standard): $-11,137.00
  MA state tax: $-4,394.80
  MA Part B exemption: $12,800.00


## Analysis

In [5]:
# Calculate the impact
ma_tax_no_ctc = sim_no_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_with_ctc = sim_with_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_change = ma_tax_with_ctc - ma_tax_no_ctc

exemption_no_ctc = sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_with_ctc = sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_change = exemption_with_ctc - exemption_no_ctc

itemizes_no_ctc = bool(sim_no_ctc.calculate('tax_unit_itemizes', 2026)[0])
itemizes_with_ctc = bool(sim_with_ctc.calculate('tax_unit_itemizes', 2026)[0])

print("\nIMPACT OF FEDERAL CTC EXPANSION:")
print(f"  MA state tax change: ${ma_tax_change:,.2f}")
print(f"  MA Part B exemption change: ${exemption_change:,.2f}")
print(f"  Itemization change: {itemizes_no_ctc} → {itemizes_with_ctc}")

if abs(ma_tax_change) > 0.01:
    print(f"\nMECHANISM:")
    print(f"1. CTC expansion triggers federal itemization")
    print(f"2. MA allows itemizers to claim medical expense deduction")
    print(f"3. Medical expense deduction: ${sim_with_ctc.calculate('medical_expense_deduction', 2026)[0]:,.2f}")
    print(f"4. MA tax rate: 5%")
    print(f"5. Tax savings: ${exemption_change:,.2f} × 5% = ${exemption_change * 0.05:,.2f}")
else:
    print("\nNo MA tax impact in this example.")


IMPACT OF FEDERAL CTC EXPANSION:
  MA state tax change: $0.00
  MA Part B exemption change: $0.00
  Itemization change: False → False

No MA tax impact in this example.
